In [21]:
import boto3
import csv
import json
import time
from datetime import datetime
from looper_scraper import scrape_looper
import datetime as dt
import pandas as pd

In [22]:
region = 'us-east-2'

streamName = 'blossom-data-eng-giftysdovie'

kinesis = boto3.client('kinesis', region_name=region)

kinesis.list_streams()

#response = kinesis.create_stream(StreamName=streamName, ShardCount=1)
kinesis.list_streams()

{'StreamNames': ['blossom-data-eng-Priscilla',
  'blossom-data-eng-clifford',
  'blossom-data-eng-dennis',
  'blossom-data-eng-emily',
  'blossom-data-eng-giftysdovie',
  'blossom-data-eng-jedidiah',
  'blossom-data-eng-michael_asante',
  'blossom-data-eng-richmond',
  'blossom-data-eng-username',
  'blossom-eng-Priscilla',
  'blossom-eng-juliuslotsu',
  'blossom-eng-michaelasante',
  'blossom-eng-omaj',
  'kojo-kinesis-test',
  'real_estate'],
 'HasMoreStreams': False,
 'ResponseMetadata': {'RequestId': 'c4dc38ae-1b4b-3750-928f-5bef2099ddf8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c4dc38ae-1b4b-3750-928f-5bef2099ddf8',
   'x-amz-id-2': '1h2N3ug615P3vcA1vuaaJFzsLDNwYwl269aaggyd0K0LpCqFbif+WLHznIqzWVxZQqpDJ6SYaelr1GFCHuVGzST6UrgJ/ooJ',
   'date': 'Sat, 23 Nov 2019 16:40:33 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '428'},
  'RetryAttempts': 0}}

In [23]:
stream_description = kinesis.describe_stream(StreamName=streamName)
stream_description.keys()
stream_description['StreamDescription']['Shards']
stream_description['StreamDescription']['Shards'][0]

{'ShardId': 'shardId-000000000000',
 'HashKeyRange': {'StartingHashKey': '0',
  'EndingHashKey': '170141183460469231731687303715884105727'},
 'SequenceNumberRange': {'StartingSequenceNumber': '49601656101500254946504064292541026311190454110666096642',
  'EndingSequenceNumber': '49601665365776178673488917553444812941182202304918454274'}}

Creating A Producer

In [40]:
def get_data(criteria):
    data = scrape_looper(1)  # ignore the argument 1
    data = data.head(2) # our assumption; first 2 listings are new
    
    # we may have some criteria to filter our data on eg "address='adenta'"
    if criteria:
        data = data.query(criteria)
    
    # for this exercise, we'll only select these columns 
    #data = data[['description', 'address', 'beds', 'showers', 'area', 'price', 'currency', 'url', 'source', 'lat', 'lon']]
    
    return data

In [41]:
def listings_producer(stream_name, data):
        response = kinesis.put_record(
            StreamName=stream_name,
            Data=data,
            PartitionKey='blossom',
        )
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"Records pushed to {streamName} within kinesis")
        else:
            print("Records not failed to be pushed to kinesis")

In [42]:
if __name__ == '__main__':
    try:
        while True:
            data = get_data(None)
            data = data.to_json()
            listings_producer(streamName, data)
            time.sleep(10)
    except Exception as e:
        print(e)

Getting page 1 from looperghana
"['lon', 'lat', 'description'] not in index"
